# Setup

In [1]:
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Cloning https://github.com/cocodataset/cocoapi.git to /private/var/folders/03/4dl3rtgd0njccwgh38g1glr400fbw2/T/pip-req-build-wzljgb5v
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /private/var/folders/03/4dl3rtgd0njccwgh38g1glr400fbw2/T/pip-req-build-wzljgb5v
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
anceled
ERROR: Operation cancelled by user


# Load Ground Truths

In [1]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2022-11-02 12:37:51--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.41.41
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.41.41|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip.1’

annotations_trainva 100%[===================>] 241.19M  33.3MB/s    in 7.7s    

2022-11-02 12:37:59 (31.5 MB/s) - ‘annotations_trainval2017.zip.1’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
!unzip -o annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  

In [1]:
import json

with open("./annotations/instances_val2017.json", "r") as f:
    ground_truths = json.load(f)

In [2]:
img_ids = [gt["image_id"] for gt in ground_truths["annotations"]][:20]

In [3]:
ground_truths["annotations"] = [
    gt for gt in ground_truths["annotations"] if gt["image_id"] in img_ids
]

In [4]:
with open("mutated_ground_truths.json", "w") as f:
    json.dump(ground_truths, f)

In [5]:
# high density version is 0.95, low density 0.0
minscore = 0.9995

In [6]:
import random


def mutate_bounding_box(bounding_box):
    def shift(x, xw):
        return xw * random.uniform(-1, 1) + x

    return [
        shift(bounding_box[0], bounding_box[2] / 10),
        shift(bounding_box[1], bounding_box[3] / 10),
        random.uniform(0.9, 1.1) * bounding_box[2],
        random.uniform(0.9, 1.1) * bounding_box[3],
    ]


results = []
for gt in [gt for gt in ground_truths["annotations"] if gt["image_id"] in img_ids]:
    result = {
        "image_id": gt["image_id"],
        "area": gt["area"],
        "bbox": mutate_bounding_box(gt["bbox"]),
        "id": gt["id"],
        "category_id": gt["category_id"],
        "score": random.uniform(minscore, 1),
    }
    results.append(result)

with open("coco_results.json", "w") as f:
    json.dump(results, f)

In [7]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open("coco_results.json", "r") as f:
    results = json.load(f)
coco = COCO("./mutated_ground_truths.json")
ret = {}

cocoDt = coco.loadRes(results)
cocoEval = COCOeval(coco, cocoDt, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.86s).
Accumulating evaluation results...
DONE (t=1.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.615
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.733
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.635
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.591
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.641
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.447
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

# Creating y_true, y_pred

In [8]:
import numpy as np
from collections import defaultdict

num_imgs = len(img_ids)

groups = defaultdict(list)

for annotation in ground_truths["annotations"]:
    img_id = annotation["image_id"]
    bounding_box = annotation["bbox"]
    bounding_box = [x for x in bounding_box] + [int(annotation["category_id"])]
    groups[img_id].append(bounding_box)

imgs = sorted(groups.keys())

result = []
for img in imgs:
    bounding_boxes = groups[img]
    bounding_boxes = np.array(bounding_boxes)
    result.append(bounding_boxes)

m = max([r.shape[0] for r in result])


def to_shape(a, shape):
    y_, x_ = shape
    y, x = a.shape
    y_pad = y_ - y
    x_pad = x_ - x
    return np.pad(a, ((0, y_pad), (0, x_pad)), mode="constant", constant_values=-1)


result = [to_shape(r, (m, 5)) for r in result]
y_true = np.array(result).astype(float)

print(y_true.shape)

(20, 25, 5)


In [ ]:
print(y_true)

In [ ]:
import numpy as np
from collections import defaultdict

num_imgs = len(img_ids)

groups = defaultdict(list)

for annotation in results:
    img_id = annotation["image_id"]
    bounding_box = annotation["bbox"]
    bounding_box = (
        [x for x in bounding_box]
        + [int(annotation["category_id"])]
        + [annotation["score"]]
    )
    groups[img_id].append(bounding_box)

imgs = sorted(groups.keys())

result = []
for img in imgs:
    bounding_boxes = groups[img]
    bounding_boxes = np.array(bounding_boxes)
    result.append(bounding_boxes)

m = max([r.shape[0] for r in result])


def to_shape(a, shape):
    y_, x_ = shape
    y, x = a.shape
    y_pad = y_ - y
    x_pad = x_ - x
    return np.pad(a, ((0, y_pad), (0, x_pad)), mode="constant", constant_values=-1)


result = [to_shape(r, (m, 6)) for r in result]
y_pred = np.array(result)

print(y_pred.shape)

In [ ]:
print(y_pred)

In [ ]:
outfile = "high_density_confidence.npz"
np.savez(outfile, y_true, y_pred)